# 1. predicting adsorption for each adsorption data point using Vt&BET only

In [4]:
import pandas as pd
import numpy as np
import os 
BET_path = 'C:/Kai_Zhang/MachineLearning/Unified gas Adsorption/Treated/Methane'

def pred_dataset(file_names):
    source_path = 'C:/Kai_Zhang/MachineLearning/Unified gas Adsorption/CO2_adsorption/new_data'
    train_df = pd.DataFrame()
    for file_name in file_names:
        temp_data = pd.read_excel(os.path.join(source_path,file_name+'-02-02-2022.xlsx'),skiprows= 1 )
        #temp_data = temp_data.dropna(axis=0,how = 'any',subset = feature_set)
        train_df = pd.concat([train_df,temp_data],axis=0)
       
    return train_df

In [110]:
from scipy import interpolate
import numpy as np
import pandas as pd

def inter_data(x_old,y_old):
    
    f = interpolate.interp1d(x_old, y_old,'slinear')    
    x_new = np.linspace(min(x_old), max(x_old),num=50)
       
    """
        for ele in x_old:
            t = bi_find(x_new,ele)

            x_new[t] = ele
        
        if (len(x_new)>150):
            x_new = x_new[len(x_new)-150:]
         = np.arange(min(x_old), max(x_old), (max(x_old)-min(x_old))/150)
    """
    y_new = f(x_new)
    #X_feature = np.append(x_new.reshape(-1,1),y_new.reshape(-1,1),axis=1).reshape(2,35).tolist()
    #x_mat = np.append(x_mat,X_feature,axis=1)
       
    return x_new, y_new

In [111]:
data = pred_dataset(["Methane"])
data.head()

,Unnamed: 0,Literature,Literature_full,Unnamed: 1,GAC,GAS,E,S,A,B,...,%N,Temp(K),Pressure,Adsorp(mmol/g),Index,logP,logQ,logD,Label,Vmic_index
0,0,1,1,26,Graphene,Methane,0,0,0,0,...,NaN,278.0,0.053415,0.018737,2751,-2.929665,-3.977255,-1.047591,Methane,0
1,1,1,1,27,Graphene,Methane,0,0,0,0,...,NaN,278.0,0.170583,0.061587,2751,-1.768532,-2.787307,-1.018775,Methane,0
2,2,1,1,28,Graphene,Methane,0,0,0,0,...,NaN,278.0,0.301536,0.104402,2751,-1.198866,-2.259508,-1.060643,Methane,0
3,3,1,1,29,Graphene,Methane,0,0,0,0,...,NaN,278.0,0.404920,0.136500,2751,-0.904066,-1.991431,-1.087365,Methane,0
4,4,1,1,30,Graphene,Methane,0,0,0,0,...,NaN,278.0,0.534150,0.171229,2751,-0.627079,-1.764751,-1.137672,Methane,0


In [112]:

BET_df = pd.DataFrame()
cols = ["E","S","A","B","V","BET","Vt","Vmic","Vmeso","Temp(K)","Pressure","Adsorp(mmol/g)","Index"]
def bet_extraction(temp_df:pd.DataFrame):
    global BET_df
    literature = temp_df["Literature"].values[0]
    gac_name = temp_df["GAC"].values[0]
    print(literature,gac_name)
    
    try:
        if literature!=np.nan:
                bet_data = BET_path+'/'+str(literature)+"/"+gac_name+".csv"
                #print(bet_data)
                bet_curve = pd.read_csv(bet_data,header=None)
                
                x = bet_curve.iloc[:,0].values.tolist()
                y = bet_curve.iloc[:,1].values.tolist()
                _,new_y = inter_data(x,y)
                rows = len(temp_df)
                bet_matrix = [new_y for _ in range(rows)]
                bet_df = pd.DataFrame(bet_matrix,columns = ["BET_"+str(i) for i in range(len(new_y))])
                bet_df = pd.concat([bet_df,temp_df[cols].reset_index()],axis= 1,ignore_index=True)
                print(bet_df.shape)
                BET_df = pd.concat([BET_df,bet_df,],axis= 0,ignore_index=True)    
               

        else:
            print("There is not leterature for this gac: {}".format(gac_name))         
    except:
        print("The BET curve for gac: {} from paper {} is not available.".format(gac_name,literature))
total_index = list(set(data["Index"].values.tolist()))
for index in total_index:
    temp_df =  data[data["Index"]==index]
    bet_extraction(temp_df)
print(i)

1 Graphene
(8, 64)
1 Graphene
(8, 64)
nan Norit R1 Extra
The BET curve for gac: Norit R1 Extra from paper nan is not available.
nan nan
The BET curve for gac: nan from paper nan is not available.
nan nan
The BET curve for gac: nan from paper nan is not available.
nan nan
The BET curve for gac: nan from paper nan is not available.
nan BPL
The BET curve for gac: BPL from paper nan is not available.
nan nan
The BET curve for gac: nan from paper nan is not available.
nan nan
The BET curve for gac: nan from paper nan is not available.
nan nan
The BET curve for gac: nan from paper nan is not available.
nan A10
The BET curve for gac: A10 from paper nan is not available.
nan nan
The BET curve for gac: nan from paper nan is not available.
nan nan
The BET curve for gac: nan from paper nan is not available.
nan nan
The BET curve for gac: nan from paper nan is not available.
nan MaxSorb
The BET curve for gac: MaxSorb from paper nan is not available.
nan nan
The BET curve for gac: nan from paper na

In [113]:
BET_df

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,101.380981,121.421652,123.031594,126.114134,129.076200,129.006998,128.937796,134.861910,141.008552,146.911738,...,0,0.2495,514.0,3.00,NaN,NaN,278.0,0.053415,0.018737,2751
1,101.380981,121.421652,123.031594,126.114134,129.076200,129.006998,128.937796,134.861910,141.008552,146.911738,...,0,0.2495,514.0,3.00,NaN,NaN,278.0,0.170583,0.061587,2751
2,101.380981,121.421652,123.031594,126.114134,129.076200,129.006998,128.937796,134.861910,141.008552,146.911738,...,0,0.2495,514.0,3.00,NaN,NaN,278.0,0.301536,0.104402,2751
3,101.380981,121.421652,123.031594,126.114134,129.076200,129.006998,128.937796,134.861910,141.008552,146.911738,...,0,0.2495,514.0,3.00,NaN,NaN,278.0,0.404920,0.136500,2751
4,101.380981,121.421652,123.031594,126.114134,129.076200,129.006998,128.937796,134.861910,141.008552,146.911738,...,0,0.2495,514.0,3.00,NaN,NaN,278.0,0.534150,0.171229,2751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5564,327.041757,399.535749,441.203175,471.788788,497.698318,520.736926,534.951482,549.166038,563.890068,579.093969,...,0,0.2495,1942.8,1.14,0.73,0.41,323.0,0.469890,0.392164,3599
5565,327.041757,399.535749,441.203175,471.788788,497.698318,520.736926,534.951482,549.166038,563.890068,579.093969,...,0,0.2495,1942.8,1.14,0.73,0.41,323.0,0.612107,0.421251,3599
5566,327.041757,399.535749,441.203175,471.788788,497.698318,520.736926,534.951482,549.166038,563.890068,579.093969,...,0,0.2495,1942.8,1.14,0.73,0.41,323.0,0.766028,0.470575,3599
5567,327.041757,399.535749,441.203175,471.788788,497.698318,520.736926,534.951482,549.166038,563.890068,579.093969,...,0,0.2495,1942.8,1.14,0.73,0.41,323.0,0.937053,0.540081,3599


In [108]:
BET_df.to_csv('./Methane_Adding_bet_80_new.csv')

In [109]:
BET_df = pd.read_csv("Methane_Adding_bet_35_new.csv")
BET_df.dropna(axis=0,how="any",subset = ["BET","Vt","Vmic","BET_34"],inplace=True)
#BET_df.dropna(axis=0,how="any",subset = ["BET_34"],inplace=True)

In [70]:
BET_df.columns

Index(['Unnamed: 0', 'BET_0', 'BET_1', 'BET_2', 'BET_3', 'BET_4', 'BET_5',
       'BET_6', 'BET_7', 'BET_8', 'BET_9', 'BET_10', 'BET_11', 'BET_12',
       'BET_13', 'BET_14', 'BET_15', 'BET_16', 'BET_17', 'BET_18', 'BET_19',
       'BET_20', 'BET_21', 'BET_22', 'BET_23', 'BET_24', 'BET_25', 'BET_26',
       'BET_27', 'BET_28', 'BET_29', 'BET_30', 'BET_31', 'BET_32', 'BET_33',
       'BET_34', 'BET_35', 'BET_36', 'BET_37', 'BET_38', 'BET_39', 'BET_40',
       'BET_41', 'BET_42', 'BET_43', 'BET_44', 'BET_45', 'BET_46', 'BET_47',
       'BET_48', 'BET_49', 'BET_50', 'BET_51', 'BET_52', 'BET_53', 'BET_54',
       'BET_55', 'BET_56', 'BET_57', 'BET_58', 'BET_59', 'BET_60', 'BET_61',
       'BET_62', 'BET_63', 'BET_64', 'BET_65', 'BET_66', 'BET_67', 'BET_68',
       'BET_69', 'BET_70', 'BET_71', 'BET_72', 'BET_73', 'BET_74', 'BET_75',
       'BET_76', 'BET_77', 'BET_78', 'BET_79', 'E', 'S', 'A', 'B', 'V', 'BET',
       'Vt', 'Vmic', 'Vmeso', 'Temp(K)', 'Pressure', 'Adsorp(mmol/g)', 'Index',


In [91]:
# data splitting
#from random import shuffle
from sklearn.model_selection import train_test_split
from numpy import random
def group_split(BET_df:pd.DataFrame,random_state = 42):
    indexs = list(set(BET_df["Index"].values.tolist()))
    #shuffle(indexs)
    _,test_index = train_test_split(indexs,test_size= 0.2,random_state =random_state)

    #test_index = random.choice(indexs,len(indexs)//5,replace=False).tolist()
    train_df = BET_df[~BET_df["Index"].isin(test_index)]
    test_df = BET_df[BET_df["Index"].isin(test_index)]
    return train_df,test_df

In [17]:
BET_df = BET_df[BET_df.notna()]

In [21]:
BET_df.describe

<bound method NDFrame.describe of       Unnamed: 0       BET_0       BET_1       BET_2       BET_3       BET_4  \
16            16   17.643244  567.318069  603.482236  630.767130  658.843833   
17            17   17.643244  567.318069  603.482236  630.767130  658.843833   
18            18   17.643244  567.318069  603.482236  630.767130  658.843833   
19            19   17.643244  567.318069  603.482236  630.767130  658.843833   
20            20   17.643244  567.318069  603.482236  630.767130  658.843833   
...          ...         ...         ...         ...         ...         ...   
3759        3759  220.194445  242.577496  252.044049  257.873076  261.377191   
3760        3760  220.194445  242.577496  252.044049  257.873076  261.377191   
3761        3761  220.194445  242.577496  252.044049  257.873076  261.377191   
3762        3762  220.194445  242.577496  252.044049  257.873076  261.377191   
3763        3763  220.194445  242.577496  252.044049  257.873076  261.377191   

     

In [92]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor,GradientBoostingRegressor,\
    BaggingRegressor,ExtraTreesRegressor,RandomForestRegressor
from lightgbm import LGBMRegressor  
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
  
n_estimators = [50,100,120,150,180,200,300]

# define different models#,
models = [
    #('SVR',SVR(max_iter=100000)),
    #('DT',DecisionTreeRegressor(random_state=42)),\
    # ('ADBR',AdaBoostRegressor(random_state=42)), 
    #("GBR",GradientBoostingRegressor(random_state=42)),\
    #('BG',BaggingRegressor(random_state=42,n_jobs=-1)),
    #('ETR',ExtraTreesRegressor(random_state=42,n_jobs=-1)),\
    #('RF',RandomForestRegressor(n_jobs=-1,random_state=42)),
    ('LGBM',LGBMRegressor(n_jobs = -1,random_state = 42)),\
    #('MLPR',MLPRegressor(random_state=42))
    #('BGLGBM',BaggingRegressor(LGBMRegressor(n_estimators = 200, n_jobs = -1,random_state = 42), random_state=42,n_jobs=-1)),\
    #('BGETR',BaggingRegressor(ExtraTreesRegressor(n_estimators = 180,random_state=42,n_jobs=6),random_state=42,n_jobs=-1))
    ]

# set search parameters grid for different models
para_grids = { 
    'SVR':{'kernel':['linear','poly','rbf','sigmoid','precomputed']},\
    'DT':{'criterion':['squared_error', 'friedman_mse', 'absolute_error', 'poisson']},\
    'ADBR':{'n_estimators':n_estimators,'learning_rate':[0.1,0.5,1,2],'loss':['linear','square','exponential']},\
    'GBR':{'n_estimators':n_estimators,'learning_rate':[0.1,0.5,1,2]},\
    'BG':{'n_estimators':[10,50,100]},\
    'ETR':{'n_estimators':n_estimators},\
    'RF':{'n_estimators':n_estimators},\
    'LGBM':{'num_leaves':[10,20,30,50,],'learning_rate': [0.05,0.1,0.5,1],'n_estimators':n_estimators},\
    'BGLGBM':{'n_estimators':[10,30,50]},\
    'BGETR':{'n_estimators':[10]},\
    'MLPR':{'hidden_layer_sizes':[(4,100),(5,100),(6,100)],
            "activation":["logistic","relu"],\
            "solver":['sgd', 'adam']}
    }

In [93]:
from sklearn.model_selection import GridSearchCV,cross_validate,GroupKFold
from sklearn.ensemble import ExtraTreesRegressor
from  sklearn.metrics import mean_squared_error,r2_score
from sklearn.utils import shuffle

def model_CV(train_x,train_y,groups,model,para_grid):

    out_cv = GroupKFold(n_splits = 5)
    result = GridSearchCV(model,para_grid,cv= out_cv.get_n_splits(groups =groups),
    scoring='neg_mean_squared_error', return_train_score=True,n_jobs=-1)
    result.fit(train_x,train_y)
    
    model_refit =model.set_params(**result.best_params_)
    train_cv = cross_validate(model_refit,train_x,train_y,groups = groups,cv =out_cv,scoring = ('r2', 'neg_mean_squared_error'))
    train_mse_cv = -train_cv['test_neg_mean_squared_error'].mean()
    train_r2_cv = train_cv['test_r2'].mean()
    
    return [train_r2_cv,train_mse_cv],result.best_params_

# model evaluation
def model_eval(model,test_x,test_y):
      
    test_pre = model.predict(test_x)
    test_r2 = r2_score(test_pre,test_y)
    test_mse = mean_squared_error(test_y,test_pre)
    return test_r2,test_mse

# comparing different models
def model_comparison(model_list,para_grids,feature_list,gas_list):
    gas_list = gas_list 
    input_feature = feature_list
    output = ['Adsorp(mmol/g)']
    result_total = []
    for gas in gas_list:
        if gas =='total':
            train_df_com = train_df
            test_df_com = test_df
            train_x = train_df_com[input_feature]
            test_x = test_df_com[input_feature]
            train_y = train_df_com[output].values
            test_y = test_df_com[output].values
            groups = train_df_com['Index'].values
            train_x, train_y, groups = shuffle(train_x, train_y, groups, random_state=42)
            
            for model_name, model in model_list:
                result, best_param = model_CV(train_x,train_y.squeeze(),groups,model,para_grids[model_name])
                model_refit = model.set_params(**best_param)
                model_refit.fit(train_x,train_y.squeeze())
                test_r2_total,test_mse_total = model_eval(model_refit,test_x,test_y.squeeze()) 
                for gases in gas_list[1:]:
                    test_df_com = test_df[test_df['Label']==gases]
                    test_xs = test_df_com[input_feature]
                    test_ys = test_df_com[output].values
                    test_r2,test_mse = model_eval(model_refit,test_xs,test_ys.squeeze()) 
                    result_total.append([gases,model_name+'_total',result[0],result[1],test_r2_total,test_mse_total,test_r2,test_mse,best_param])
                    print('Dataset {}, Algorithm {}, Test_r2 {}, Test_error {}'.format(gas,model_name+'_total',test_r2,test_mse))    
        else:
            train_df_com = train_df[train_df['Label']==gas]
            test_df_com = test_df[test_df['Label']==gas]
            train_x = train_df_com[input_feature]
            test_x = test_df_com[input_feature]
            train_y = train_df_com[output].values
            test_y = test_df_com[output].values
            groups = train_df_com['Index']
            train_x, train_y, groups = shuffle(train_x, train_y, groups, random_state=42)
            for model_name, model in model_list:
                result, best_param = model_CV(train_x,train_y.squeeze(),groups,model,para_grids[model_name])
                model_refit = model.set_params(**best_param)
                model_refit.fit(train_x,train_y.squeeze())
                test_r2,test_mse = model_eval(model_refit,test_x,test_y.squeeze()) 
                result_total.append([gas,model_name+'_separate',result[0],result[1],-1,-1, test_r2,test_mse,best_param])
                
                print('Dataset {}, Algorithm {}, Test_r2 {}, Test_error {}'.format(gas,model_name+'_total',test_r2,test_mse))     
    return result_total

In [94]:
import os
conditions = ['S','V','Temp(K)','Pressure']
base_feature = ["BET_"+str(i) for i in range(35)]+conditions
feature_list = [base_feature,base_feature+["BET"],base_feature+['Vt'],base_feature+["BET",'Vt'],\
    base_feature+["BET","Vt","Vmic","Vmeso"],["BET","Vt"]+conditions,["BET","Vt","Vmic","Vmeso"]+conditions]
columns = ['Gas','Model_name','CV_r2','CV_mse','test_r2_total_model','test_mse_by_total_model','test_r2_separa_model','test_mse_separa_model','best_param']
file_name = ['BET_Curve',"BET_Curve_BET","BET_Cureve_Vt","BET_Curve_BET_Vt","BET_Curve_BET_Vt_Vmic_Vmeso","BET_Vt","BET_Vt_Vmic_Vmeso"] #['BET_Curve','BET_Plus_Vt',"Vmic_Vmeso"]
gas_list = ['Methane']#,'CFCs','Methane','E&E'

for i in range(50): 
    train_df,test_df = group_split(BET_df,random_state=i)
    for j in range(len(feature_list)):
    
        results = model_comparison(models,para_grids, feature_list[j],gas_list)
        files_name = '35_point_curve_feature_'+file_name[j]+'_result_'+str(i)+'.csv'
        pd.DataFrame(results,columns = columns).to_csv(os.path.join('./8_using_BET_curve',files_name)) 
    print("The finish of {} th trial".format(i)) 

Dataset Methane, Algorithm LGBM_total, Test_r2 0.9447608429629457, Test_error 0.5892927033619536
Dataset Methane, Algorithm LGBM_total, Test_r2 0.9374708025540808, Test_error 0.6661730790202247
Dataset Methane, Algorithm LGBM_total, Test_r2 0.9432400141170167, Test_error 0.5973685588553374
Dataset Methane, Algorithm LGBM_total, Test_r2 0.9404942978595685, Test_error 0.6358956576399813
Dataset Methane, Algorithm LGBM_total, Test_r2 0.9463210072441994, Test_error 0.5748451631443579
Dataset Methane, Algorithm LGBM_total, Test_r2 0.8870572181769175, Test_error 1.1830155432924234
Dataset Methane, Algorithm LGBM_total, Test_r2 0.9266936180381943, Test_error 0.7552002574422811
The finish of 0 th trial
Dataset Methane, Algorithm LGBM_total, Test_r2 0.9102770287673082, Test_error 0.8630933136509413
Dataset Methane, Algorithm LGBM_total, Test_r2 0.9345064246444277, Test_error 0.5982954890666641
Dataset Methane, Algorithm LGBM_total, Test_r2 0.9320313870258796, Test_error 0.6584886980771499
Datas

# Post result treatments
 

In [95]:
import pandas as pd
file_name = ['BET_Curve',"BET_Curve_BET","BET_Cureve_Vt","BET_Curve_BET_Vt","BET_Curve_BET_Vt_Vmic_Vmeso","BET_Vt","BET_Vt_Vmic_Vmeso"] 

cal_columns= ["CV_r2","CV_mse","test_r2_separa_model","test_mse_separa_model"]
for j in range(len(file_name)):
    df_list = []
    for i in range(50):
        files_name = '35_point_curve_feature_'+file_name[j]+'_result_'+str(i)+'.csv'
        df_list.append(pd.read_csv(os.path.join('./8_using_BET_curve',files_name))[cal_columns] )
        pd.concat(df_list).groupby(level=0).mean().to_csv(os.path.join('./8_using_BET_curve',file_name[j]+'_mean.csv'))
        pd.concat(df_list).groupby(level=0).std().to_csv(os.path.join('./8_using_BET_curve',file_name[j]+'_std.csv'))

# 2_Predicting adsorption for each data point using the combination Vt, BET, Vmeso, and Vmic
the dataset for each separate gas will be smaller than previous one.

In [ ]:
import os
base_feature = ['V','S','L','BET',]
condition_feature = ['Temp(K)','Pressure']
combin_1 = ['Vt']
combin_2 = ["Vmeso"]
combin_3 = ['Vmic']
combin_4 = ['Vt',"Vmeso",]
combin_5 = ['Vt',"Vmic",]
combin_6 = ['Vt',"Vmic",'Vmeso',]
combin_7 = ["Vmic",'Vmeso',]

feature_list = [base_feature+condition_feature,base_feature+combin_1+condition_feature, \
    base_feature+combin_3+condition_feature, base_feature+combin_2+condition_feature,\
    base_feature+combin_4+condition_feature, base_feature+combin_5+condition_feature,\
    base_feature+combin_6+condition_feature, base_feature+combin_7+condition_feature, ]

columns = ['Gas','Model_name','CV_r2','CV_mse','test_r2_total_model','test_mse_by_total_model','test_r2_separa_model','test_mse_separa_model','best_param']


feature_set = ["BET","Vt","Vmeso","Vmic"]
gas_list = ['CO2','CFCs','Methane','E&E']
file_name = ['BET_only','BET_plut_Vt',"BET_Vmic","BET_Vmeso",'BET_Vt_Vmeso','BET_Vt_Vmic',"BET_Vt_Vmic_meso","BET_Vmic_meso"]


for i in range(10,15):
    train_df,test_df = pred_dataset(['CO2','Methane','Ethane&Ethylene','CFCs'],feature_set= feature_set)
    for j in range(len(feature_list)):
        results = model_comparison(models,para_grids, feature_list[j],gas_list)
        files_name = 'Four_gases_with_Vmic_'+file_name[j]+'_result_'+str(i)+'.csv'
        pd.DataFrame(results,columns = columns).to_csv(os.path.join('./2_Predicting_separate_gas_BET_Vt_Vmeso_Vmic',files_name))  
        #pd.DataFrame(results,columns = ['Gas','Algo','Train_erro','Test_error']).to_csv(os.path.join('./',files_name))   

# Post result treatment

In [24]:
import pandas as pd
file_name = ['BET_only','BET_plut_Vt',"BET_Vmic","BET_Vmeso",'BET_Vt_Vmeso','BET_Vt_Vmic',"BET_Vt_Vmic_meso","BET_Vmic_meso"]

cal_columns= ["CV_r2","CV_mse","test_r2_separa_model","test_mse_separa_model"]
for j in range(len(file_name)):
    df_list = []
    for i in range(11):
    
        files_name = 'Four_gases_with_Vmic_'+file_name[j]+'_result_'+str(i)+'.csv'
        df_list.append(pd.read_csv(os.path.join('./2_Predicting_separate_gas_BET_Vt_Vmeso_Vmic',files_name))[cal_columns] )
        pd.concat(df_list).groupby(level=0).mean().to_csv(os.path.join('./2_Predicting_separate_gas_BET_Vt_Vmeso_Vmic',file_name[j]+'_mean_new.csv'))
        pd.concat(df_list).groupby(level=0).std().to_csv(os.path.join('./2_Predicting_separate_gas_BET_Vt_Vmeso_Vmic',file_name[j]+'_std_new.csv'))


# Predicting the fitted parameters of adsorption isotherms

# Using only BET and Vt

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def pred_dataset(file_names, feature_set = feature_set):
    source_path = 'C:/Kai_Zhang/MachineLearning/Unified gas Adsorption/CO2_adsorption/new_data'
    train_df = pd.DataFrame()
    test_df = pd.DataFrame()
    for file_name in file_names:
        temp_data = pd.read_excel(os.path.join(source_path,file_name+'-fitting-02-02-2022.xlsx'),skiprows= 1 )
        temp_data = temp_data.dropna(axis=0,how = 'any',subset = feature_set)
        train_x,test_x = train_test_split(temp_data,test_size = 0.2)
        train_df = pd.concat([train_df,train_x],axis=0)
        test_df = pd.concat([test_df,test_x],axis =0)
    return train_df,test_df

In [ ]:
from sklearn.model_selection import GridSearchCV,cross_validate,GroupKFold
from sklearn.ensemble import ExtraTreesRegressor
from  sklearn.metrics import mean_squared_error,r2_score
from sklearn.utils import shuffle

def model_CV(train_x,train_y,model,para_grid):

    
    result = GridSearchCV(model,para_grid,cv= 5,
    scoring='neg_mean_squared_error', return_train_score=True,n_jobs=-1)
    result.fit(train_x,train_y)
    
    model_refit =model.set_params(**result.best_params_)
    train_cv = cross_validate(model_refit,train_x,train_y,cv =5,scoring = ('r2', 'neg_mean_squared_error'))
    train_mse_cv = -train_cv['test_neg_mean_squared_error'].mean()
    train_r2_cv = train_cv['test_r2'].mean()
    
    return [train_r2_cv,train_mse_cv],result.best_params_

# model evaluation
def model_eval(model,test_x,test_y):
      
    test_pre = model.predict(test_x)
    test_r2 = r2_score(test_pre,test_y)
    test_mse = mean_squared_error(test_y,test_pre)
    return test_r2,test_mse

# comparing different models
def model_comparison(model_list,para_grids,feature_list,gas_list):
    gas_list = gas_list 
    input_feature = feature_list
    output = ['Adsorp(mmol/g)']
    result_total = []

    for gas in gas_list:
        
        if gas =='total':

            train_df_com = train_df
            test_df_com = test_df
            train_x = train_df_com[input_feature]
            test_x = test_df_com[input_feature]
            train_y = train_df_com[output].values
            test_y = test_df_com[output].values
            
            train_x, train_y = shuffle(train_x, train_y,random_state=42)
            
            for model_name, model in model_list:

                
                result, best_param = model_CV(train_x,train_y.squeeze(),model,para_grids[model_name])
                model_refit = model.set_params(**best_param)
                model_refit.fit(train_x,train_y.squeeze())
                test_r2_total,test_mse_total = model_eval(model_refit,test_x,test_y.squeeze()) 
                for gases in gas_list[1:]:
                    test_df_com = test_df[test_df['Label']==gases]
                    test_xs = test_df_com[input_feature]
                    test_ys = test_df_com[output].values
                    test_r2,test_mse = model_eval(model_refit,test_xs,test_ys.squeeze()) 
                    result_total.append([gases,model_name+'_total',result[0],result[1],test_r2_total,test_mse_total,test_r2,test_mse,best_param])

                    print('Dataset {}, Algorithm {}, Test_r2 {}, Test_error {}'.format(gas,model_name+'_total',test_r2,test_mse))

            
        else:
            
            train_df_com = train_df[train_df['Label']==gas]
            test_df_com = test_df[test_df['Label']==gas]
            train_x = train_df_com[input_feature]
            test_x = test_df_com[input_feature]
            train_y = train_df_com[output].values
            test_y = test_df_com[output].values
            
            train_x, train_y = shuffle(train_x, train_y, random_state=42)
           
            for model_name, model in model_list:

                result, best_param = model_CV(train_x,train_y.squeeze(),model,para_grids[model_name])
                model_refit = model.set_params(**best_param)
                model_refit.fit(train_x,train_y.squeeze())
                test_r2,test_mse = model_eval(model_refit,test_x,test_y.squeeze()) 
                result_total.append([gas,model_name+'_separate',result[0],result[1],-1,-1, test_r2,test_mse,best_param])
                
                print(print('Dataset {}, Algorithm {}, Test_r2 {}, Test_error {}'.format(gas,model_name+'_total',test_r2,test_mse))
)
                
    return result_total